In [200]:
#%run gw_exp.ipynb
# import functions from gw_exp.ipynb

In [201]:
def kontsevich(d):
    """
    Returns the number of rational curves of degree d passing through 3d-1 
    general points in P^2.
    These numbers are also the genus zero Gromov-Witten invariants for P^2. 
    
    EXAMPLES::
    
        sage: kontsevich(3)
        12
        sage: kontsevich(4)
        620
        sage: kontsevich(5)
        87304
    """
    if d == 1:
        return 1
    if d == 2:
        return 1
    f = 0
    for i in range(1,d):
        j = d - i
        a = kontsevich(i)
        b = kontsevich(j)
        f = f + i**2*j*(j*binomial(3*d-4,3*i-2)-i*binomial(3*d-4,3*i-1))*a*b
    return f

def bipart(l):
    """
    Needs this to compute genus zero Gromow-Witten invariants for projective 
    spaces.
    EXAMPLES::
        sage: bipart([1,2])  
        [[[2], [1]], [[1, 2], []], [[], [1, 2]], [[1], [2]]]
        sage: len(bipart([1,2]))
        4
        sage: type(bipart([1,2]))
        <type 'list'>
    """
    if len(l) == 0:
        return [[[],[]]]
    elif len(l) == 1:
        k = l[0]
        return [[[],[k]],[[k],[]]]
    else:
        ll = [l[i] for i in range(len(l)-1)]
        R = bipart(ll)
        R1 = [[R[i][0] + [l[-1]], R[i][1]] for i in range(len(R))]
        R2 = [[R[i][0], R[i][1] + [l[-1]]] for i in range(len(R))]
        return R1 + R2

@CachedFunction
def GW_invariant_helper(dim, d, l):
        """
        Returns the Gromov-Witten invariants for this projective space.
        INPUT: 
            d is the degree
            l is a list of non-negative integers: for example the number 3
            denotes the class h^3.
        OUTPUT:
            The number of degree d curves meeting some general subvarieties in
            this projective space.
        EXAMPLES::
            Computing the number of lines meeting 4 general lines in P^3.
            sage: P = ProjectiveSpace(3)
            sage: P.GW_invariant(1,[2,2,2,2])
            2
            Computing the number of conics meeting 8 general lines in P^3.
            sage: P = ProjectiveSpace(3)
            sage: P.GW_invariant(2,[2]*8)            
            92
        """
        r = dim
        #l.sort()
        #l.reverse()
        n = len(l)
        for i in l:
            if i > r:
                return 0
            pass
        if sum(l) != r*d + r + d + n - 3:
            return 0
        elif n == 0 or n == 1:
            return 0
        elif d == 0:
            if n == 3 and sum(l) == r:
                return 1
            else:
                return 0
        elif n == 2:
            if d == 1 and l == (r,r):
                return 1
            else:
                return 0
        else:
            if l[-1] == 0:
                return 0
            elif l[-1] == 1:
                return d*GW_invariant(dim, d,l[:-1])
            elif r == 2:
                if l == [2]*(3*d-1):
                    return kontsevich(d)
                else:
                    return 0
            else:
                l1 = l[-1] - 1
                l2 = 1
                ll = [l2] + [l[i] for i in range(n-2)] + [l1+l[n-2]]
                res = GW_invariant(dim, d,ll)
                S = bipart([l[i] for i in range(n-3)])
                for s in S:
                    A = tuple(s[0])
                    nA = len(A)
                    cA = sum(A)
                    B = tuple(s[1])
                    for dA in range(1,d+1):
                        dB = d - dA
                        e = r*dA + r + dA + nA - cA - l1 - l[n-2]
                        if e >= 0 and e <= r:
                            a = GW_invariant(dim, dA,(l1,l[n-2])+tuple(A)+(e,))
                            b = GW_invariant(dim, dB,(l2,l[n-3])+tuple(B)+(r-e,))
                            res = res + a*b
                        f = r*dA + r + dA + nA - cA - l1 - l2
                        if f >= 0 and f <= r:
                            x = GW_invariant(dim, dA,(l1,l2)+tuple(A)+(f,))
                            y = GW_invariant(dim, dB,(l[n-2],l[n-3])+tuple(B)+(r-f,))
                            res = res - x*y
                return res

def GW_invariant(n, b, A):
    A_  = sorted(A)
    A_.reverse()
    return GW_invariant_helper(n, b, tuple(A_))


In [202]:
print(bipart([1,1]))

[[[1], [1]], [[1, 1], []], [[], [1, 1]], [[1], [1]]]


In [203]:
class ClassList:
    # DO NOT USE THE FOLLOWING METHODS
    def __init__(self, c_list):
        self.n = len(c_list) - 1  # corresponding dimension of projective space
        self.c_list = c_list
        self.contain_wrong_class = False
 
    def __add__(self, c_list):
        if isinstance(c_list, type(self)):
            return ClassList((c1 + c2 for c1, c2 in zip(self.c_list, c_list)))
        elif isinstance(c_list, list):
            # assuming adding list of classes of the form
            # [1,1,1,2,2,2,...]
            new_c_list = list(self.c_list)
            for c in c_list:
                if not (c >= 0 and c <= self.n):
                    self.contain_wrong_class = True
                    continue
                new_c_list[c] += 1
            return ClassList(new_c_list)
        else:
            raise TypeError("Invalid argument type: {}".format(type(c_list)))


    def __radd__(self, c_list):
        return self.__add__(c_list)
    
    def __getitem__(self, key):
        if isinstance(key, type(0)) or isinstance(key, int): # check if given sage integer as index
            if key < 0:
                key = self.__len__() + key
            curr_key = 0

            for c, count in enumerate(self.c_list):
                curr_key += count
                if curr_key > key and count > 0:
                    return c

            raise IndexError("List index out of range")

        elif isinstance(key, slice):
            # assume given a slice object in this case
            # a slice object has attribute start, stop, step
            # not handling step for now
            start, stop, step = key.indices(len(self))
            
            new_c_list = [0] * (self.n+1) #list(self.c_list)
            curr_key = 0

            for c, count in enumerate(self.c_list):
                if count == 0:
                    continue

                curr_key += count

                if curr_key > start:
                    if curr_key - start <= count:
                        new_c_list[c] = curr_key - start
                    else:
                        new_c_list[c] = count
                    
                if curr_key >= stop:
                    if (curr_key - start) <= count:
                        new_c_list[c] = stop - start
                    else:
                        new_c_list[c] = count - (curr_key - stop)
                    break

            return ClassList(new_c_list)
                    
        else:
            raise TypeError("Invalid argument type: {}".format(type(key)))

    def __setitem__(self, key, item):
        curr_key = 0
        for c, count in self.c_list:
            if curr_key > key:
                if count > 0:
                    self.c_list[c] -= 1
                self.c_list[item] += 1
                return
            curr_key += count
        raise IndexError("List index out of range")

    def __delitem__(self, key):
        print("delete itrm")
        curr_key = 0
        for c, count in self.c_list:
            if curr_key > key:
                if count > 0:
                    self.c_list[c] -= 1
                else:
                    return
            curr_key += count
        raise IndexError("List index out of range")

    def __iter__(self):
        return self.decompress().__iter__()

    def __hash__(self):
        return tuple(self.c_list).__hash__()

    def __eq__(self, other):
        if isinstance(other, list):
            return ClassList.compress(other, self.n).c_list == self.c_list
        elif isinstance(other, ClassList):
            return other.c_list == self.c_list
        else:
            raise TypeError(f"Invalid argument type: {type(other)}")

    def __len__(self):
        return sum(self.c_list)

    def __str__(self):
        return str(self.c_list)
    
    # YOU MAY USE THE FOLLOWING METHODS


    # deleting by index of class list of the form [1,1,1,2,2,2...]
    # return a new ClassList with the class deleted
    def remove_class(self, c):
        new_c_list = list(self.c_list)
        new_c_list[c] -= 1
        return ClassList(new_c_list)
    
    # find the minimum class
    def min(self):
        for c, count in enumerate(self.c_list):
            if count > 0:
                return c
        return -1
    
    @staticmethod
    def compress(l, n):
        c_list = [0]*(n+1) 
        for c in l:
            c_list[c] += 1
        return ClassList(c_list)
        
    def decompress(self):
        return ClassList.decompress_(self.c_list)

    @staticmethod
    def decompress_(c_list):
        l = []
        for c, count in enumerate(c_list):
            l += [c]*count
        return l

    # this is actually not real bipartition
    def bipart(self):
        def helper(left, right, index):
            if index == len(self.c_list) - 1:
                for current in range(self.c_list[index] + 1):
                    left[index] = current
                    right[index] = self.c_list[index] - current
                    yield ClassList(left), ClassList(right)
            else:
                for current in range(self.c_list[index] + 1):
                    left[index] = current
                    right[index] = self.c_list[index] - current
                    yield from helper(left, right, index + 1)

        yield from helper(
            [0] * len(self.c_list),
            [0] * len(self.c_list),
            0)



In [204]:
l = ClassList([0, 2, 0, 0, 0])
l_ = [3, 4] + l
print([1,1,3,4] == l_)

True


In [205]:
def bipart_coeff(I1, I2):
    return product([binomial(I1.c_list[j], I2.c_list[j]) for j in range(0, I1.n+1)])

In [206]:
# ogw_exp computes the open gromov witten invariant of (CP^n, RP^n)
# Input:
# n - dimension of the real projective space RP^n
# A - list of cohomology classes represented by integers, ranging from 1...n, and -1 representing \Gamma_{square}
# b - beta, a non-negative integer
# k - number of boundary marked points, a non-negative integer
@CachedFunction
def ogw_invariant_helper(n, b, k, A_t):
    A = list(A_t)
    l = len(A)
    A.sort()    # asceding order
    #A.reverse() # to descending order

    for c_d in A:
        if c_d > n:
            return 0

    #if A.contain_wrong_class:
    #    return 0

    # wall-crossing theorem (Elad)
    if l > 0 and A[0] == -1:
        return -ogw_invariant(n, b, k+1, A[:-1])

    #degree axiom (Elad)
    if 2*sum(A) != (n - 3 + (n+1)*b + k + 2*l - k*n):  
        # The maslov index is twice the chern class n+1 for holomorphic spheres. 
        # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
        # p 595 mcduff 
        #ogw_d[str((n, b, k, A)) + ", degree axiom"] = 0
        return 0

    # other
    if k == 0 and is_even(b):
        return 0

    # zero axiom
    if b == 0:
        if k == 1 and l == 1 and A[0] == 0:
            return -1
        else:
            return 0

    # unit axiom (May)
    # handle all beta = 0 cases
    if l > 0 and A[0] == 0:
        if b == 0 and k == 1 and l == 1:
            return -1
        else:
            return 0

    # divisor axiom 
    if l > 0 and A[0] == 1:
        #A.pop(0)
        result = (b/2)*ogw_invariant(n, b, k, A[1:])
        return result

    # handle beta = 1, l <= 1 cases
    """
    if b == 1 and k == 2 and A == []:
        return 2
    if b == 1 and len(A) == 1 and k == 1:
        return 0

    if b == 1 and len(A) == 1 and k == 0:
        return (-1)^((n-1)/2)
        """
    if b == 1 and k == 0 and A == [n]:
        return (-1)^((n+3)/2)

    if b == 1 and k == 1 and A == [(n+1)/2]:
        return 0

    if b == 1 and k == 2 and A == []:
        return 2


    if l >= 2:
        cor = ogw_invariant(n, b, k, [A[0]-1, A[1]+1] + A[2:])
        col = 0
        ool = 0
        oor = 0

        for t in bipart(A[2:]): #A[2:].bipart():
            I1 = t[0]
            I2 = t[1]

            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 2 == 0:
                    b_ = (b - b1) / 2
                else:
                    continue

                for i in range(0, n+1):
                    cor += (-1)^(b_*(n+1)/2) *  GW_invariant(n, b_, [1, A[1], i] + I1) \
                        *  ogw_invariant(n, b1, k, [n-i, A[0]-1] + I2) 


            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 2 == 0:
                    b_ = (b - b1) / 2
                else:
                    continue

                for i in range(0, n+1):
                    col += (-1)^((n+1)*b_/2)* GW_invariant(n, b_, [1, A[0] - 1, i] + I1) \
                        * ogw_invariant(n, b1, k, [n-i, A[1]] + I2)

            
            for b1 in range(1, b):
                b2 = b - b1

                for k1 in range(0, k+1):
                    k2 = k - k1
                    ool += binomial(k, k1)*ogw_invariant(n, b1, k1, [1, A[0]-1] + I1) \
                        * ogw_invariant(n, b2, k2 + 1, [A[1]] + I2)

            
            for b1 in range(1, b):
                b2 = b - b1

                for k1 in range(0, k+1):
                    k2 = k - k1

                    oor += binomial(k, k1) * ogw_invariant(n, b1, k1, [1, A[1]] + I1) \
                        * ogw_invariant(n, b2, k2 + 1, [A[0] - 1] + I2)

        return cor - col - oor + ool

    if k >= 2:
        result = 0
        co = 0
        ool = 0
        oor = 0


        for t in bipart(A):
            I1 = t[0]
            I2 = t[1]

            for b1 in range(1, b+1):
                b_ = 0
                if (b + 1 - b1) % 2 == 0:
                    b_ = (b + 1 - b1) / 2
                else:
                    continue

                for i in range(0, n+1):
                    co += (-1)^((n+1)*b_/2)*GW_invariant(n, b_, [1, n, i] + I1) \
                        * ogw_invariant(n, b1, k-1, [n-i] + I2)                   
            
            for b1 in range(2, b+1):
                b2 = b + 1 - b1

                for k1 in range(0, (k - 2)+1):
                    k2 = k - 2 - k1
                    ool += binomial(k-2, k1)*ogw_invariant(n, b1, k1, [1, n] + I1)\
                        * ogw_invariant(n, b2, k2+2, I2)

            for b1 in range(1, b+1):
                b2 = b + 1 - b1

                for k1 in range(0, (k - 2)+1):
                    k2 = k - 2 - k1
                    oor += binomial(k-2, k1) * ogw_invariant(n, b1, k1 + 1, [1] + I1) \
                        * ogw_invariant(n, b2, k2+1, [n]+I2)

        return (-1)^((n+3)/2)*2*(co - ool + oor)

    print("unexpected situation")
    print("n: ", n, ", b: ", b, ", k: ", k, " A: ", A)
    return 0

def ogw_invariant(n, b, k, A):
    return ogw_invariant_helper(n, b, k, tuple(A))



In [208]:
print(ogw_invariant(3, 5, 10, []))
#print(ogw_invariant(5, 3, 0, [2]*7+[4]*1))


-2
